In [4]:
import xml.etree.ElementTree as ET
import requests
import psycopg2
import os
from dotenv import load_dotenv
from lxml import etree

# Umgebungsvariablen laden (sollten identisch zu den anderen Skripts sein)
load_dotenv()
# Fetch the XML data from the URL
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/lfsa_esgais/1.0?compress=false"
response = requests.get(url)
xml_data = response.text if response.status_code == 200 else None

# Define namespaces 
namespaces = {
    'm': 'http://www.sdmx.org/resources/sdmxml/schemas/v3_0/message',

}

# Parse the XML data
root = ET.fromstring(xml_data)

# Find the <m:DataSet> element
data_set = root.find('.//m:DataSet', namespaces)

# Extracting data from XML file -> shorten XML file and format first ~20 lines to get a look at structure
structured_data = []
for series in data_set.findall('.//Series', namespaces):  #
    series_data = series.attrib
    for obs in series.findall('.//Obs', namespaces):
        if 'OBS_VALUE' in obs.attrib and obs.attrib['OBS_VALUE']:
            obs_data = obs.attrib  # 
            record = {**series_data, **obs_data}  
            structured_data.append(record)




# raw_data
data = structured_data

dbname = 'Ismene'
user = os.getenv('DB_USER', 'DB_USER')
password = os.getenv('DB_PASSWORD', 'DB_PASSWORD')
host = os.getenv('DB_HOST', 'DB_HOST')

# Connect 
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, options = "-c search_path=Source")
cur = conn.cursor()

# I do not know how to change the table_name to file name whcih would be easier imho
table_name = '"Source".self_employment_by_occupation_raw'

# SQL to create a table if it doesn't exist
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    age TEXT,
    freq TEXT,
    geo TEXT,
    isco08 TEXT,
    sex TEXT,
    unit TEXT,
    wstatus TEXT,
    time_period TEXT,
    obs_value NUMERIC,
    obs_flag TEXT
);
"""


cur.execute(create_table_query)


insert_stmt = f"""
INSERT INTO {table_name} (age, freq, geo, isco08, sex, unit, wstatus, time_period, obs_value, obs_flag)
VALUES (%(age)s, %(freq)s, %(geo)s, %(isco08)s, %(sex)s, %(unit)s, %(wstatus)s, %(TIME_PERIOD)s, %(OBS_VALUE)s, %(OBS_FLAG)s);
"""


for record in data:
    # define dafault value for missing data
    record_with_defaults = {
        'age': record.get('age'),
        'freq': record.get('freq'),
        'geo': record.get('geo'),
        'isco08': record.get('isco08'),
        'sex': record.get('sex'),
        'unit': record.get('unit'),
        'wstatus': record.get('wstatus'),
        'TIME_PERIOD': record.get('TIME_PERIOD'),
        'OBS_VALUE': record.get('OBS_VALUE'),
        'OBS_FLAG': record.get('OBS_FLAG', 'na') # Provide a default value
    }
    cur.execute(insert_stmt, record_with_defaults)


conn.commit()


cur.close()
conn.close()
